In [11]:
def neg(fact):
    return -fact
    
class Fact:
    def __init__(self, text, neg=False):
        self.text = text
        self.negation = neg
        
    def copy(self):
        return Fact(self.text, self.negation)
        
    def __repr__(self):
        if self.negation:
            return '!' + self.text
        else:
            return self.text
        
    def __str__(self):
        return self.__repr__()
    
    def __neg__(self):
        neg_fact = self.copy()
        neg_fact.negation = !neg_fact.negation
        return neg_fact
    
    def __and__(self, other):
        if isinstance(other, Fact):
            return Rule(self, other)
        
        elif isinstance(other, Rule):
            other.addFact(other)
        
        else:
            raise TypeError
            
    def __eq__(self, other):
        if isinstance(other, Fact):
            return other.text == self.text and other.negation == self.negation
        
        return False
    
    def __hash__(self):
        return hash(self.text) #+ hash(self.negation)

class Rule:
    def __init__(self, f1, f2=None):
        self.facts = [f1, f2]
        self.product_fact = None
        self.active = True
        
    def addFact(self, fact):
        self.facts.append(fact)
        
    def copy(self):
        copy_rule = Rule(self.facts[0], self.facts[1])
        if(len(self.facts) > 2):
            for i in range(2, len(self.facts)):
                copy_rule.addFact(self.facts[i])
        return copy_rule
    
    def give(self, fact):
        self.product_fact = fact
        return self
        
    def __str__(self):
        return self.__repr__()
        
    def __repr__(self):
        s = ''
        for f in self.facts:
            s += str(f) + ' & '
            
        return s[:-3] + ' => ' + str(self.product_fact)
        
    def __and__(self, other):
        if isinstance(other, Fact):
            new_rule = self.copy()
            new_rule.addFact(other)
            return new_rule
        
        elif isinstance(other, Rule):
            new_rule = self.copy()
            for f in other.facts:
                new_rule.addFact(f)
                
            return new_rule

        else:
            raise TypeError

In [12]:
class AkiNN():
    def __init__(self):
        self.facts = set()
        self.givenFacts = set()
        self.rules = []
        
    def addFact(self, text):
        self.facts.append(Fact(text))
        
    def addRule(self, rule):
        for f in rule.facts:
            self.facts.add(f)
            
        self.facts.add(rule.product_fact)
            
        self.rules.append(rule)
        
    def _getRulesForFact(self, fact):
        l = []
        for r in self.rules:
            if fact == r.product_fact:
                l.append(r)
                
        return l
    
    def _ask(self, fact):
        print(f'Est-ce que {fact} ?')
        
    def _propagateRule(self, rule, given_facts):
        empty = True
        ret = False
        for pre in rule.facts:
            print("evaluating ", pre)
            if pre != given_facts:
                empty = False
                break
        if empty:
            print("Propagating rule " + str(rule.pre) + " => " + str(rule.post))
            for post in rule.product_fact:
                if post != given_facts:
                    if self.verb: print("Adding " + post + " as a new fact")
                    given_facts(post)
                    ret = True # At least one fact has been added
            rule.active = False
        return ret
        
    def ForwardChaining(self, given_facts):
        "Simple Forward Checking algorithm. No smart data structure used"
        loop = True # True if any fact has been added
        while loop:
            loop = False
            for rule in self.rules:
                if rule.active:
                    loop |= self._propagateRule(rule, given_facts)
        
    def BackwardChaining(self, fact, given_facts):
        "Simple Backward chaining for a fact, returns after any new fact is added after any question"
        print("BC for fact " + str(fact))
        for rule in self._getRulesForFact(fact):
            print(rule)
            for pre in rule.facts:
                if pre not in given_facts:
                    rulespre = self._getRulesForFact(pre)
                    if not rulespre: # no rules conclude on it. This is an askable fact
                        self._ask(pre)
                        return True
                    else:
                        return self.BackwardChaining(pre)

# AkiNN
## Base de faits

In [13]:
system = AkiNN()

# Ce qu'on veut traiter
in_img = Fact('entree-image')
in_son = Fact('entree-son')
in_phrase = Fact('entree-phrase')
in_txt = Fact('entree-texte')
in_mot = Fact('entree-mot')
in_video = Fact('entree-video')
in_vec = Fact('entree-vecteur')
in_mat = Fact('entree-matrice')

temp = Fact('temporalite')

# Ce qu'on veut faire
act_class = Fact('action-classification')
act_seg = Fact('action-segmentation')
act_reg = Fact('action-regression')
act_compl = Fact('action-completion')
act_compr = Fact('action-compression')
act_gen = Fact('action-generation')
act_feat = Fact('action-features')

gen_like = Fact('generation-ressemble')
gen_new = Fact('generation-nouveau')

rec_sup = Fact('reconstruction-elevee')

# Ce qu'on a
bdd_label = Fact('bdd-etiquetee')
bdd_unlabel = Fact('bdd-non-etiquetee')

class_bin = Fact('classes-binaire')
class_mul = Fact('classes-multiple')

# Ce qu'on peut obtenir
nn_perc = Fact('reseau-perceptron')
nn_mlp = Fact('reseau-mlp')
nn_cnn = Fact('reseau-cnn')
nn_cnn_mlp = Fact('reseau-cnn+mlp')
nn_rnn = Fact('reseau-rnn')
nn_rnn_mlp = Fact('reseau-rnn+mlp')
nn_hopf = Fact('reseau-hopf')
nn_bolt = Fact('reseau-boltzmann')
nn_ae = Fact('reseau-autoencoder')
nn_vae = Fact('reseau-vae')
nn_gan = Fact('reseau-gan')

beta_high = Fact('parametre-beta-grand')
beta_low = Fact('parametre-beta-petit')

## Bases de règles

In [19]:
system.addRule((in_img & temp).give(in_video))

system.addRule((in_img & act_class).give(nn_cnn_mlp))
system.addRule((in_img & act_class).give(nn_mlp))
system.addRule((in_img & act_seg).give(nn_cnn))
system.addRule((in_img & act_compl).give(nn_rnn))

system.addRule((in_img & act_compr).give(nn_vae))
system.addRule((in_img & act_gen & rec_sup).give(nn_vae))
system.addRule(Rule(rec_sup).give(beta_low))
system.addRule((in_img & act_gen & neg(rec_sup)).give(nn_gan))
system.addRule((in_img & act_gen & neg(rec_sup)).give(nn_vae))
system.addRule(Rule(neg(rec_sup)).give(beta_high))

system.addRule((in_son & act_class).give(nn_cnn_mlp))
system.addRule((in_son & act_class).give(nn_mlp))
system.addRule((in_son & act_seg).give(nn_cnn))
system.addRule((in_son & act_seg).give(nn_rnn))
system.addRule((in_son & act_compr).give(nn_vae))
system.addRule((in_son & act_compl).give(nn_rnn))
system.addRule((in_son & act_gen).give(nn_rnn))
system.addRule((in_son & act_gen).give(nn_gan))

system.addRule((in_phrase & act_class).give(nn_mlp))
system.addRule((in_phrase & act_class).give(nn_cnn))
system.addRule((in_phrase & act_compl).give(nn_rnn))
system.addRule((in_phrase & act_gen).give(nn_rnn))
system.addRule((in_phrase & act_gen).give(nn_gan))

system.addRule((in_txt & act_class).give(nn_rnn))
system.addRule((in_txt & act_class).give(nn_cnn))
system.addRule((in_txt & act_seg).give(nn_rnn))
system.addRule((in_txt & act_seg).give(nn_cnn))
system.addRule((in_txt & act_compl).give(nn_rnn))
system.addRule((in_txt & act_compr).give(nn_rnn))
system.addRule((in_txt & act_gen).give(nn_rnn))

system.addRule((in_video & act_class).give(nn_rnn))
system.addRule((in_video & act_class).give(nn_cnn))
system.addRule((in_video & act_seg).give(nn_rnn))
system.addRule((in_video & act_compl).give(nn_cnn))
system.addRule((in_video & act_compl).give(nn_gan))
system.addRule((in_video & act_compr).give(nn_rnn))
system.addRule((in_video & act_gen).give(nn_gan))

system.addRule(Rule(in_mat).give(in_vec))
system.addRule((in_vec & act_reg).give(nn_rnn))
system.addRule((in_vec & act_class).give(nn_cnn))
system.addRule((in_vec & act_seg).give(nn_rnn))
system.addRule((in_vec & act_compl).give(nn_cnn))
system.addRule((in_vec & act_compl).give(nn_gan))
system.addRule((in_vec & act_compr).give(nn_rnn))
system.addRule((in_vec & act_gen).give(nn_gan))

In [7]:
facts = [in_img]
system.BackwardChaining(nn_rnn, facts)

BC for fact reseau-rnn
entree-image & action-classification & temporalite => reseau-rnn
Est-ce que action-classification ?


True